## [TTA정보통신용어사전](https://terms.tta.or.kr/main.do) 용어목록 스크래퍼
 - TTA에서 제공하는 용어를 스크래핑하는 프로그램

In [ ]:
import selenium.webdriver
from selenium.webdriver import Chrome
from selenium.webdriver import ChromeOptions
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, NoSuchElementException,StaleElementReferenceException

import time
import atexit
import os
import pandas as pd
from glob import glob

import re
from urllib import parse

In [ ]:
#전역 변수 선언
TIMEOUT = 10
ROOT_DIR = path.dirname(__file__) if "__file__" in locals() else os.getcwd()
DOWNLOAD_DIR = f'{ROOT_DIR}\\download\\tta'
print(DOWNLOAD_DIR)

#사이트 주소
URL = 'http://terms.tta.or.kr/dictionary/searchFirstList.do'

In [ ]:
#chrome webdriver를 생성한다.
# - download_path: 파일을 다운로드 받을 위치
# - headless: True이면 브라우저 화면 없이 background로 처리
def create_chrome(download_path, headless=False) :
    options = ChromeOptions()
    if headless:
        options.add_argument('headless')
        options.add_argument('--disable-gpu')
    options.add_experimental_option('prefs', {
        'download.default_directory' : download_path,
        'download.prompt_for_download' : False
    })

    chrome = Chrome(chrome_options = options)

    if headless :
        _enable_download_on_chrome_headless(chrome, download_path)

    #프로세스 종료시 크롬을 종료시키는 로직
    atexit.register(_close_chrome(chrome))
    
    return chrome

#headless=True 일때 다운로드 안되는(크롬 보안문제) 문제 해결
def _enable_download_on_chrome_headless(browser, download_dir) :
    browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')

    params = {
        'cmd': 'Page.setDownloadBehavior',
        'params': {
            'behavior': 'allow',
            'downloadPath': download_dir
        }
    }
    browser.execute("send_command", params)

#크롬 종료~
def _close_chrome(chrome) :
    def close() :
        chrome.close()
    return close

def chrome_wait(wait, condition):
    try:
        element = wait.until(condition)
        return element
    except TimeoutException:
        print(f"[DEBUG] 페이지 로딩 시간 초과")

In [ ]:
# 페이지에서 용어를 찾고, 용어, 동의어, 설명을 추출하여 리턴한다.
def find_word(e) :
    value = e.get_attribute('value').strip()
    browser.execute_script(f'javascript:fn_dictionaryView("{value}")')
    
    chrome_wait(wait, EC.presence_of_element_located((By.XPATH, f'//*[@id="cont"]/input[@name="word_seq" and @value="{value}"]')))

    time.sleep(0.3)
    word = browser.find_element_by_xpath('//div[@id="cont"]/dl/dt[1]').text
    print(f"[{value}] - word:{word}")
    synonym = ""
    try :
        synonym = browser.find_element_by_xpath('//div[@id="cont"]/dl/dt[2]').text
    except NoSuchElementException :
        print("No Such synonym..")
    
    description = browser.find_element_by_xpath('//div[@id="cont"]/dl/dd[1]/div').get_attribute('innerHTML')
    description = re.sub('<.+?>', '', description, 0, re.I|re.S)

    return word, synonym, description

In [ ]:
#크롬 브라우저 생성 및 첫번째 카테고리를 선택
browser = create_chrome(download_path=DOWNLOAD_DIR)
wait = WebDriverWait(browser, TIMEOUT)
browser.get(URL)
browser.execute_script("javascript:fn_firstWord('ㄱ')")

In [ ]:
#카테고리를 자동으로 이동하기 위해 전체 카테고리정보를 가져온다.
categories = browser.find_elements_by_xpath('//p[@class="korean" or @class="english"]/a')
categories = [parse.unquote(category.get_attribute('href')) for category in categories]

In [ ]:
#카테고리 별 페이지 이동하여 용어를 찾아 수집한다.
for category in categories :
    tta_words = []

    print(f"[INFO] Category: {category}")
    browser.execute_script(category)
    start_page = 0
    chrome_wait(wait, EC.presence_of_element_located((By.XPATH, '//a[@href="{category}" and @class="on"]')))

    value = browser.find_element_by_xpath('//li/div[starts-with(text(), "[총")]').text
    total_count = int(re.findall("\d+", value)[0])
    total_page = total_count // 10 + (total_count % 10 > 0 and 1 or 0)
    print(f"[INFO] 다운로드 전체 페이지: {total_page}")

    for page in range(start_page, total_page) : #page
        print(f'[INFO]{page + 1} 처리')
        if page != start_page :
            browser.execute_script(f'javascript:fnNavigate({page + 1});')

        chrome_wait(wait, EC.presence_of_element_located((By.NAME, 'first_seq')))
        elements = browser.find_elements_by_name('first_seq')
        for element in elements :
            word, synonym, description = find_word(element)
            tta_words.append([word, synonym, description])
    df = pd.DataFrame(tta_words, columns=["word", "synonym", "description"])

    filename = category.replace("javascript:fn_firstWord('", "").replace("')", "")
    df.to_csv(os.path.join(DOWNLOAD_DIR, f"tta_dictionay({filename}).csv"), index=False, encoding="utf-8")

#수집된 데이터 DataFrame으로 만든 후 CSV 파일에 저장